# Imports

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
# map
import folium
from branca.element import Template, MacroElement

# Constants

In [78]:
data_location = "/content/gdrive/MyDrive/First Year 2022-23/Spring 1 9-5 1 2023/NSC 325 Invester Program Energy/arXiv/data"

# Data Processing

In [130]:
df = pd.read_excel(f"{data_location}/UT_Comp_Seq.xlsx")

## Dataset Information (don't run)

In [ ]:
df[df.isna().any(axis=1)]

,Operator,Well Name,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",...,Best3 Mo BOPD,Best6 Mo BOPD,Best9 Mo BOPD,Best12 Mo BOPD,Compl. Type,Latitude,Longitude,Fluid Type from DI,SPACING_CAPPED,12 month Cum Prod
14,Operator 11,Well 163,MBH,9587.0,NaN,160,92,35,4464678.0,61643.0,...,267.433706,233.497699,216.962525,203.9119,Sleeves,48.634007,-102.529256,SW-XL HYBRID,660.000000,73.91
20,Operator 32,Well 2366,TFH,9975.0,30.0,146,94,1,3516320.0,66375.0,...,516.020162,352.613412,260.965008,NaN,Sleeves,47.492298,-102.593452,SW-XL HYBRID,660.000000,84.80
21,Operator 7,Well 2457,TFH,8776.0,32.0,144,97,33,2604477.0,98399.0,...,257.955293,236.094675,NaN,NaN,Sleeves,47.242609,-102.985050,SW-XL HYBRID,660.000000,69.92
22,Operator 35,Well 2476,TFH,9186.0,46.0,148,94,33,4683157.0,113311.0,...,683.870261,455.840456,328.241654,NaN,P & P,47.589299,-102.670506,SW-XL HYBRID,378.192380,130.67
23,Operator 6,Well 2646,MBH,8739.0,34.0,145,97,7,4015504.0,73370.0,...,NaN,NaN,NaN,NaN,P & P (cmt.),47.387980,-103.097102,SW-XL HYBRID,660.000000,96.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10851,Operator 4,Well 10829,MBH,9419.0,25.0,159,97,5,2823700.0,80438.0,...,NaN,NaN,NaN,NaN,P & P (cmt.),48.632490,-103.237726,NaN,660.000000,28.65
10852,Operator 18,Well 10834,TFH,9685.0,50.0,155,96,24,5002417.0,70777.0,...,NaN,NaN,NaN,NaN,Sleeves,48.238376,-102.965904,NaN,390.130820,109.08
10853,Operator 4,Well 10837,MBH,9717.0,37.0,155,99,33,6628250.0,180943.0,...,NaN,NaN,NaN,NaN,P & P (cmt.),48.199224,-103.422342,NaN,520.504035,202.44
10854,Operator 18,Well 10839,MBH,9750.0,35.0,155,96,24,3490684.0,4829.0,...,NaN,NaN,NaN,NaN,Sleeves,48.238377,-102.966040,NaN,255.220160,56.58


In [ ]:
# check if a column contains floating point numbers
col = 'Fluid, bbls'
df[col][df[col].notna() & (df[col] % 1 != 0)]

0        31638.333333
43       64192.166667
44       61749.357143
45       55731.166667
46       53488.095238
             ...     
10501    25164.333333
10623    38527.452381
10654    67823.928571
10671    39766.666667
10721    42082.309524
Name: Fluid, bbls, Length: 943, dtype: float64

In [ ]:
df.head(2)

,Operator,Well Name,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",...,Best3 Mo BOPD,Best6 Mo BOPD,Best9 Mo BOPD,Best12 Mo BOPD,Compl. Type,Latitude,Longitude,Fluid Type from DI,SPACING_CAPPED,12 month Cum Prod
0,Operator 7,Well 92,MBH,9707.0,21.0,143,98,35,1283184.0,31638.333333,...,134.330484,119.937541,103.824238,91.288626,OH,47.155062,-103.057148,UNKNOWN,660.0,34.41
1,Operator 7,Well 288,TFH,8990.0,21.0,160,91,5,2094358.0,57618.000000,...,109.478413,102.602455,92.523194,85.796077,OH,48.717103,-102.452273,UNKNOWN,660.0,19.19


In [ ]:
for col in df:
  print(col)

Operator
Well Name
Formation
Lateral Length
Stages
Township 
Range
Section
Total Prop, lbs
Fluid, bbls
Date Fracd
Best1 Mo BOPD
Best3 Mo BOPD
Best6 Mo BOPD
Best9 Mo BOPD
Best12 Mo BOPD
Compl. Type
Latitude
Longitude
Fluid Type from DI
SPACING_CAPPED
12 month Cum Prod


In [ ]:
for col in df:
  print(df[col].value_counts())

Operator 18    1235
Operator 1     1201
Operator 4      999
Operator 2      786
Operator 12     672
Operator 11     641
Operator 32     546
Operator 41     545
Operator 30     528
Operator 22     341
Operator 36     334
Operator 24     321
Operator 38     278
Operator 7      264
Operator 35     249
Operator 23     242
Operator 6      221
Operator 34     215
Operator 27     140
Operator 50     138
Operator 33     131
Operator 20     117
Operator 45     113
Operator 46     102
Operator 29      93
Operator 26      54
Operator 10      54
Operator 25      50
Operator 37      39
Operator 44      34
Operator 13      33
Operator 15      22
Operator 48      17
Operator 19      17
Operator 51       9
Operator 43       9
Operator 21       8
Operator 28       7
Operator 9        6
Operator 8        6
Operator 5        6
Operator 17       5
Operator 49       5
Operator 39       5
Operator 55       4
Operator 56       2
Operator 16       2
Operator 53       2
Operator 47       1
Operator 3        1


In [ ]:
for col in df:
  print(col)
  temp = (df[col].unique())
  temp = list(map(str, temp))
  temp.sort()
  print(temp[0:2], temp[-2:])

Operator
['Operator 1', 'Operator 10'] ['Operator 8', 'Operator 9']
Well Name
['Well 1', 'Well 10'] ['Well 9998', 'Well 9999']
Formation
['MBH', 'MBH/TFH'] ['UTFH', 'nan']
Lateral Length
['10000.0', '10001.0'] ['9999.0', 'nan']
Stages
['1.0', '10.0'] ['94.0', 'nan']
Township 
['139', '140'] ['163', '164']
Range
['100', '101'] ['98', '99']
Section
['1', '10'] ['8', '9']
Total Prop, lbs
['0.0', '1000000.0'] ['9995280.0', '9999525.0']
Fluid, bbls
['0.0', '100004.0'] ['9990.0', 'nan']
Date Fracd
['2004-01-01T00:00:00.000000000', '2005-01-01T00:00:00.000000000'] ['2017-11-17T00:00:00.000000000', 'NaT']
Best1 Mo BOPD
['1.87376725838264', '100.065746219592'] ['999.211045364891', 'nan']
Best3 Mo BOPD
['1.87376725838264', '100.032873109796'] ['998.882314266929', 'nan']
Best6 Mo BOPD
['10.3933815472277', '100.093140477756'] ['997.781065088757', 'nan']
Best9 Mo BOPD
['100.014610271021', '100.021915406531'] ['99.9013806706114', 'nan']
Best12 Mo BOPD
['10.7111549419242', '10.9111330265176'] ['99.95

In [ ]:
df.apply(lambda x: (x.isna().sum()/len(x)).round(1))

Operator              0.0
Well Name             0.0
Formation             0.0
Lateral Length        0.0
Stages                0.0
Township              0.0
Range                 0.0
Section               0.0
Total Prop, lbs       0.0
Fluid, bbls           0.0
Date Fracd            0.0
Best1 Mo BOPD         0.1
Best3 Mo BOPD         0.2
Best6 Mo BOPD         0.2
Best9 Mo BOPD         0.2
Best12 Mo BOPD        0.3
Compl. Type           0.0
Latitude              0.0
Longitude             0.0
Fluid Type from DI    0.5
SPACING_CAPPED        0.0
12 month Cum Prod     0.0
dtype: float64

## Missing values

In [131]:
missing_values = {
    0.0 : ["Fluid, bbls", "Total Prop, lbs"],
    "No Data" : ["Compl. Type"],
    'unknown, probably hybrid' : ["Compl. Type"],
    'Not indicated' : ["Compl. Type"],
    "UNKNOWN": ["Fluid Type from DI"],
}

In [132]:
for key in missing_values:
  cols = missing_values[key]
  df[cols] = df[cols].replace(key, np.nan)

In [133]:
df.head(1)

,Operator,Well Name,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",...,Best3 Mo BOPD,Best6 Mo BOPD,Best9 Mo BOPD,Best12 Mo BOPD,Compl. Type,Latitude,Longitude,Fluid Type from DI,SPACING_CAPPED,12 month Cum Prod
0,Operator 7,Well 92,MBH,9707.0,21.0,143,98,35,1283184.0,31638.333333,...,134.330484,119.937541,103.824238,91.288626,OH,47.155062,-103.057148,NaN,660.0,34.41


## Drop Columns: Well Name and others
Future work: I could first fill in the missing values if the operator only misses a few

In [134]:
cols_drop = ['Well Name', 'Best1 Mo BOPD', 'Best3 Mo BOPD', 'Best6 Mo BOPD', 'Best9 Mo BOPD', 'Best12 Mo BOPD', 'Fluid Type from DI']
df = df.drop(columns=cols_drop)

In [135]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
0,Operator 7,MBH,9707.0,21.0,143,98,35,1283184.0,31638.333333,2012-06-11,OH,47.155062,-103.057148,660.0,34.41


## Sort the df by Operator, Date Fracd

In [136]:
df['Operator_Num'] = df['Operator'].str.extract('(\d+)').astype(float)
df = df.sort_values(by=['Operator_Num', 'Date Fracd'])
df = df.drop(columns=['Operator_Num'])

In [137]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
5325,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,2004-01-01,NaN,48.765628,-103.03031,660.0,21.95


## Missing values per Operator

In [138]:
operator_mv = df.groupby('Operator').apply(lambda x: (x.isna().sum()/len(x)).round(1))#.sum(axis=1)

In [139]:
cols_selected = operator_mv.columns[operator_mv.sum() > 0]
operator_mv = operator_mv[cols_selected]

In [140]:
operator_mv[operator_mv.sum(axis=1) != 0]

,Stages,"Total Prop, lbs","Fluid, bbls",Compl. Type
Operator,,,,
Operator 1,0.0,0.0,0.0,0.1
Operator 14,1.0,0.0,1.0,1.0
Operator 18,0.0,0.0,0.0,0.1
Operator 2,0.0,0.0,0.0,0.1
Operator 20,0.0,0.0,0.0,0.2
Operator 21,0.0,0.0,0.0,0.2
Operator 22,0.0,0.0,0.0,0.1
Operator 23,0.0,0.0,0.1,0.0
Operator 29,0.1,0.2,0.3,0.5


In [141]:
temp = dict()

# Iterate through rows
for index, row in operator_mv.iterrows():
    # Check if any value in the row is equal to 1.0
    if any(value == 1 for value in row.values):
        temp[index] = True
    else:
        temp[index] = False

# Create a boolean mask
mask = [temp[index] for index in operator_mv.index]

# Apply the mask to the DataFrame
operator_mv = operator_mv[mask]
operator_mv.index


Index(['Operator 14', 'Operator 3', 'Operator 31', 'Operator 42',
       'Operator 52', 'Operator 53', 'Operator 54', 'Operator 55'],
      dtype='object', name='Operator')

In [142]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
5325,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,2004-01-01,NaN,48.765628,-103.03031,660.0,21.95


## Drop Operators

In [143]:
operators_drop = list(operator_mv.index)
# Drop rows where 'Operator' column matches the names in operators_to_drop
df = df[~df['Operator'].isin(operators_drop)]

In [144]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
5325,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,2004-01-01,NaN,48.765628,-103.03031,660.0,21.95


## Reset Row Index

In [145]:
df = df.reset_index()
df = df.drop(columns='index')

In [146]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
0,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,2004-01-01,NaN,48.765628,-103.03031,660.0,21.95


## Completion Type

In [147]:
df['Compl. Type'] = df['Compl. Type'].replace('P & P (cmt.)', 'P & P')
df['Compl. Type'] = df['Compl. Type'].replace('P & P (After sleeves mill out)', 'P & P')
df['Compl. Type'] = df['Compl. Type'].replace('P & P (After milling out sleeves)', 'P & P')

In [148]:
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Sleeves & \d+ P&P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Sleeves & \d+ P & P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Sleeves and \d+ P & P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Sleeves and \d+ P&P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Sleeve and \d+ P & P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ sleeve and \d+ P & P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ sleeves and \d+ P & P', 'Sleeves and P & P', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace('Sleeves and P & P (cmt.)', 'Sleeves and P & P')

In [149]:
df['Compl. Type'] = df['Compl. Type'].replace('Sleeves (cmt.)', 'Sleeves')

In [150]:
df['Compl. Type'] = df['Compl. Type'].replace('Frac ports', 'Frac Ports')

In [151]:
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Frac Ports and \d+ P & P', 'Frac Ports and P & P', regex=True)

In [152]:
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Frac Ports and \d+ Repeater Ports', 'Frac Ports and Repeater Ports', regex=True)
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Frac Port \d+ Repeater Ports', 'Frac Ports and Repeater Ports', regex=True)

In [153]:
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ Repeater Ports and \d+ P & P', 'Repeater Ports and P & P', regex=True)

In [154]:
df['Compl. Type'] = df['Compl. Type'].replace(r'\d+ P & P & \d+ CT annulus', 'CT and P & P', regex=True)

In [155]:
df['Compl. Type'].unique()

array([nan, 'P & P', 'Sleeves and P & P', 'CT and P & P', 'OH', 'Sleeves',
       'CT', 'Screen', 'Cemented Liner', 'Frac Ports and P & P',
       'Frac Ports and Repeater Ports', 'Frac Ports',
       'Repeater Ports and P & P', 'Perforated Liner', 'Coil Tubing Frac'],
      dtype=object)

In [156]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod
0,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,2004-01-01,NaN,48.765628,-103.03031,660.0,21.95


## Date Fracd

In [157]:
df['Year Fracd'] = df['Date Fracd'].dt.year.astype("Int64")
df['Month Fracd'] = df['Date Fracd'].dt.month.astype("Int64")
df['Day Fracd'] = df['Date Fracd'].dt.day.astype("Int64")

In [158]:
df['Date Fracd'] = df['Year Fracd'] * 10000 + df['Month Fracd'] * 100 + df['Day Fracd']

In [159]:
df.head(1)

,Operator,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Compl. Type,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod,Year Fracd,Month Fracd,Day Fracd
0,Operator 1,MBH,4376.0,1.0,161,95,17,319000.0,NaN,20040101,NaN,48.765628,-103.03031,660.0,21.95,2004,1,1


## Encode Categorical Values - One Hot Encoding (not ordinal)

In [160]:
df['Operator'] = df['Operator'].str.extract('(\d+)').astype("Int64")

In [161]:
categorical_cols = df.select_dtypes(exclude=['number']).columns

In [162]:
categorical_cols

Index(['Formation', 'Compl. Type'], dtype='object')

In [163]:
# Convert categorical columns to numeric using one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols)

In [164]:
df_encoded.head(1)

,Operator,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Latitude,...,Compl. Type_Frac Ports,Compl. Type_Frac Ports and P & P,Compl. Type_Frac Ports and Repeater Ports,Compl. Type_OH,Compl. Type_P & P,Compl. Type_Perforated Liner,Compl. Type_Repeater Ports and P & P,Compl. Type_Screen,Compl. Type_Sleeves,Compl. Type_Sleeves and P & P
0,1,4376.0,1.0,161,95,17,319000.0,NaN,20040101,48.765628,...,0,0,0,0,0,0,0,0,0,0


## Fill in Missing Values

In [165]:
cols = df_encoded.columns
cols

Index(['Operator', 'Lateral Length', 'Stages', 'Township ', 'Range', 'Section',
       'Total Prop, lbs', 'Fluid, bbls', 'Date Fracd', 'Latitude', 'Longitude',
       'SPACING_CAPPED', '12 month Cum Prod', 'Year Fracd', 'Month Fracd',
       'Day Fracd', 'Formation_MBH', 'Formation_MBH/TFH', 'Formation_MTFH',
       'Formation_TF1', 'Formation_TF2', 'Formation_TF2.5', 'Formation_TF3',
       'Formation_TF4', 'Formation_TFH', 'Formation_TFSH', 'Formation_UTFH',
       'Compl. Type_CT', 'Compl. Type_CT and P & P',
       'Compl. Type_Cemented Liner', 'Compl. Type_Coil Tubing Frac',
       'Compl. Type_Frac Ports', 'Compl. Type_Frac Ports and P & P',
       'Compl. Type_Frac Ports and Repeater Ports', 'Compl. Type_OH',
       'Compl. Type_P & P', 'Compl. Type_Perforated Liner',
       'Compl. Type_Repeater Ports and P & P', 'Compl. Type_Screen',
       'Compl. Type_Sleeves', 'Compl. Type_Sleeves and P & P'],
      dtype='object')

In [166]:
# Initialize an empty DataFrame to store the results
df_imputed = pd.DataFrame()

# Group by 'Operator' and apply k-NN imputation for each group
for _, group in df_encoded.groupby('Operator'):
    # Create a copy of the group to avoid modifying the original DataFrame
    group_copy = group.copy()

    # Apply k-NN imputation to the numerical columns
    imputer = KNNImputer(n_neighbors=5)  # Adjust the number of neighbors as needed
    group_copy[cols] = imputer.fit_transform(group_copy[cols])

    # Append the imputed group to the result DataFrame
    df_imputed = pd.concat([df_imputed, group_copy])

# Verify the changes
df_imputed.columns

Index(['Operator', 'Lateral Length', 'Stages', 'Township ', 'Range', 'Section',
       'Total Prop, lbs', 'Fluid, bbls', 'Date Fracd', 'Latitude', 'Longitude',
       'SPACING_CAPPED', '12 month Cum Prod', 'Year Fracd', 'Month Fracd',
       'Day Fracd', 'Formation_MBH', 'Formation_MBH/TFH', 'Formation_MTFH',
       'Formation_TF1', 'Formation_TF2', 'Formation_TF2.5', 'Formation_TF3',
       'Formation_TF4', 'Formation_TFH', 'Formation_TFSH', 'Formation_UTFH',
       'Compl. Type_CT', 'Compl. Type_CT and P & P',
       'Compl. Type_Cemented Liner', 'Compl. Type_Coil Tubing Frac',
       'Compl. Type_Frac Ports', 'Compl. Type_Frac Ports and P & P',
       'Compl. Type_Frac Ports and Repeater Ports', 'Compl. Type_OH',
       'Compl. Type_P & P', 'Compl. Type_Perforated Liner',
       'Compl. Type_Repeater Ports and P & P', 'Compl. Type_Screen',
       'Compl. Type_Sleeves', 'Compl. Type_Sleeves and P & P'],
      dtype='object')

In [167]:
df_copy = df_imputed.copy()  # If you want to keep the original encoding

# If using one-hot encoding, inverse transform to original categorical columns
for col in categorical_cols:
    original_col = col.split('_')[0]  # Extract the original column name
    df_copy[original_col] = df_imputed.filter(regex=f"{original_col}_").idxmax(axis=1)

df_copy.head()

,Operator,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Latitude,...,Compl. Type_Frac Ports and Repeater Ports,Compl. Type_OH,Compl. Type_P & P,Compl. Type_Perforated Liner,Compl. Type_Repeater Ports and P & P,Compl. Type_Screen,Compl. Type_Sleeves,Compl. Type_Sleeves and P & P,Formation,Compl. Type
0,1.0,4376.0,1.0,161.0,95.0,17.0,319000.0,24163.400000,20040101.0,48.765628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Formation_MBH,Compl. Type_CT
1,1.0,6048.0,30.0,140.0,97.0,5.0,640000.0,9568.000000,20040101.0,46.964407,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Formation_MBH,Compl. Type_CT
2,1.0,4113.0,1.0,162.0,96.0,9.0,359000.0,22211.638095,20050101.0,48.878756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Formation_MBH,Compl. Type_CT
3,1.0,7194.0,1.0,162.0,95.0,7.0,787000.0,9355.000000,20050101.0,48.866062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Formation_MBH/TFH,Compl. Type_CT
4,1.0,5114.0,30.0,141.0,99.0,10.0,520764.0,16079.000000,20060101.0,47.051150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Formation_MBH,Compl. Type_CT


In [168]:
df_copy['Formation'] = df_copy['Formation'].str.replace(r'\w+_', '')

<ipython-input-168-2dd5eb7de0c1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy['Formation'] = df_copy['Formation'].str.replace(r'\w+_', '')


In [169]:
df_copy['Compl. Type'] = df_copy['Compl. Type'].str.replace('Compl. Type_', '')

<ipython-input-169-5cf2afe69e8c>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy['Compl. Type'] = df_copy['Compl. Type'].str.replace('Compl. Type_', '')


In [170]:
df_copy.columns

Index(['Operator', 'Lateral Length', 'Stages', 'Township ', 'Range', 'Section',
       'Total Prop, lbs', 'Fluid, bbls', 'Date Fracd', 'Latitude', 'Longitude',
       'SPACING_CAPPED', '12 month Cum Prod', 'Year Fracd', 'Month Fracd',
       'Day Fracd', 'Formation_MBH', 'Formation_MBH/TFH', 'Formation_MTFH',
       'Formation_TF1', 'Formation_TF2', 'Formation_TF2.5', 'Formation_TF3',
       'Formation_TF4', 'Formation_TFH', 'Formation_TFSH', 'Formation_UTFH',
       'Compl. Type_CT', 'Compl. Type_CT and P & P',
       'Compl. Type_Cemented Liner', 'Compl. Type_Coil Tubing Frac',
       'Compl. Type_Frac Ports', 'Compl. Type_Frac Ports and P & P',
       'Compl. Type_Frac Ports and Repeater Ports', 'Compl. Type_OH',
       'Compl. Type_P & P', 'Compl. Type_Perforated Liner',
       'Compl. Type_Repeater Ports and P & P', 'Compl. Type_Screen',
       'Compl. Type_Sleeves', 'Compl. Type_Sleeves and P & P', 'Formation',
       'Compl. Type'],
      dtype='object')

In [171]:
# Drop the encoded columns
drops = ['Formation_MBH',
       'Formation_MBH/TFH', 'Formation_MTFH', 'Formation_TF1', 'Formation_TF2',
       'Formation_TF2.5', 'Formation_TF3', 'Formation_TF4', 'Formation_TFH',
       'Formation_TFSH', 'Formation_UTFH', 'Compl. Type_CT',
       'Compl. Type_CT and P & P', 'Compl. Type_Cemented Liner',
       'Compl. Type_Coil Tubing Frac', 'Compl. Type_Frac Ports',
       'Compl. Type_Frac Ports and P & P',
       'Compl. Type_Frac Ports and Repeater Ports', 'Compl. Type_OH',
       'Compl. Type_P & P', 'Compl. Type_Perforated Liner',
       'Compl. Type_Repeater Ports and P & P', 'Compl. Type_Screen',
       'Compl. Type_Sleeves', 'Compl. Type_Sleeves and P & P']
df_copy.drop(columns=drops, inplace=True)

In [172]:
df_copy.columns

Index(['Operator', 'Lateral Length', 'Stages', 'Township ', 'Range', 'Section',
       'Total Prop, lbs', 'Fluid, bbls', 'Date Fracd', 'Latitude', 'Longitude',
       'SPACING_CAPPED', '12 month Cum Prod', 'Year Fracd', 'Month Fracd',
       'Day Fracd', 'Formation', 'Compl. Type'],
      dtype='object')

## Final Process

In [173]:
# convert int columns from float to int
int_cols = ['Operator', 'Stages', 'Township ', 'Range', 'Section', 'Year Fracd', 'Month Fracd', 'Day Fracd']
for col in int_cols:
  df_copy[col] = df_copy[col].astype(int)

In [174]:
df_copy['Operator'] = 'Operator ' + df_copy['Operator'].astype(str)

In [175]:
df_copy

,Operator,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Date Fracd,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod,Year Fracd,Month Fracd,Day Fracd,Formation,Compl. Type
0,Operator 1,4376.0,1,161,95,17,319000.0,24163.400000,20040101.0,48.765628,-103.030310,660.00000,21.95,2004,1,1,MBH,CT
1,Operator 1,6048.0,30,140,97,5,640000.0,9568.000000,20040101.0,46.964407,-102.956575,660.00000,12.44,2004,1,1,MBH,CT
2,Operator 1,4113.0,1,162,96,9,359000.0,22211.638095,20050101.0,48.878756,-103.152480,660.00000,27.33,2005,1,1,MBH,CT
3,Operator 1,7194.0,1,162,95,7,787000.0,9355.000000,20050101.0,48.866062,-103.054106,660.00000,12.18,2005,1,1,MBH/TFH,CT
4,Operator 1,5114.0,30,141,99,10,520764.0,16079.000000,20060101.0,47.051150,-103.214580,660.00000,33.55,2006,1,1,MBH,CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10839,Operator 51,8900.0,26,155,91,11,2600000.0,57265.000000,20130630.0,48.255722,-102.345290,660.00000,98.94,2013,6,30,MBH,OH
10840,Operator 51,8960.0,29,155,91,10,3300000.0,53658.000000,20140519.0,48.255560,-102.367197,534.02462,106.49,2014,5,19,MBH,OH
10841,Operator 51,9498.0,32,155,91,10,5800000.0,84404.000000,20140528.0,48.255561,-102.367608,660.00000,122.00,2014,5,28,MBH,P & P
10842,Operator 56,9038.0,31,154,100,8,4040615.0,79566.000000,20161115.0,48.171096,-103.580672,629.00860,174.85,2016,11,15,MBH,P & P


In [176]:
df_copy.to_csv(f"{data_location}/UT_Comp_Seq_clean.csv")

# Map Plot

In [ ]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
  });
  </script>
</head>
<body>
  <div id='maplegend' class='maplegend'
      style='position: absolute; z-index:9999; border:0px solid grey; background-color:rgba(255, 255, 255, 0.0);
        border-radius:50%; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
    <div class='legend-title'></div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li style='font-weight: bold;'><span style='background:#CD5C5C; opacity: 1; border-radius: 50%;'></span>Sample Wells</li>
      </ul>
    </div>
  </div>
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
  }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
  }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
  }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 14px;
    width: 14px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    border-radius: 50%;
  }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
  }
  .maplegend a {
    color: #777;
  }
</style>

{% endmacro %}


"""

In [ ]:
m = folium.Map(location=[48, -103], zoom_start=8)

# Add a small circle for each data point
for index, row in df.iterrows():
   folium.Circle(location=[row['Latitude'], row['Longitude']], radius=0.01, color='indianred', fill=True, fill_color='indianred').add_to(m)

# Add a compass rose (direction rose)
compass_icon = folium.CustomIcon(icon_image=f"{data_location}/NorthArrow.png", icon_size=(50, 50))
compass_marker = folium.Marker(location=[48.9, -104.3], icon=compass_icon, draggable=True)
compass_marker.add_to(m)

# legend
macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
m.save("map.html")